**M ARISH**

**SC25M159**

Optimization Techniques Algorithms

 **6. Rank-2 Update (BFGS)**

In [1]:
import numpy as np

def bfgs_rank2_quadratic(Q, b, x0, tol=1e-8, max_iter=1000, enable_history=True):
    """
    Robust BFGS (rank-2) inverse-Hessian update for
    f(x) = 0.5 * (x^T Q x - x^T b), g(x) = Qx - 0.5 b.
    Uses Armijo backtracking line search.
    """
    x = x0.astype(float)
    n = len(x)
    H = np.eye(n)
    g = Q @ x - 0.5 * b
    history = [] if enable_history else None

    def f(xv):
        return 0.5 * (xv @ Q @ xv - xv @ b)

    def line_search(xv, pv, gv, alpha_init=1.0, c=1e-4, rho=0.5, alpha_min=1e-12):
        alpha = alpha_init
        fx = f(xv)
        descent = gv @ pv
        if descent >= 0:  # not a descent direction, fallback
            pv = -gv
            descent = gv @ pv
        while f(xv + alpha * pv) > fx + c * alpha * descent:
            alpha *= rho
            if alpha < alpha_min:
                return None, pv
        return alpha, pv

    for k in range(max_iter):
        gnorm = np.linalg.norm(g)
        if gnorm < tol:
            return x, k, f(x), history

        # Search direction
        p = -H @ g

        # Line search
        alpha, p = line_search(x, p, g)
        if alpha is None:
            break

        # Step
        x_new = x + alpha * p
        g_new = Q @ x_new - 0.5 * b

        s = x_new - x
        y = g_new - g
        ys = y @ s

        # BFGS update with safeguard
        if ys > 1e-12:
            rho = 1.0 / ys
            I = np.eye(n)
            H = (I - rho * np.outer(s, y)) @ H @ (I - rho * np.outer(y, s)) + rho * np.outer(s, s)

        # Update state
        x, g = x_new, g_new
        if enable_history:
            history.append(x.copy())

        # Reset H if it becomes unstable
        if not np.isfinite(H).all():
            H = np.eye(n)

    return x, max_iter, f(x), history


# Problem data
Q = np.array([[3, 0, 1],
              [0, 4, 2],
              [1, 2, 3]], dtype=float)
b = np.array([3, 0, 1], dtype=float)

x0 = np.array([0.0, 0.0, 0.0])

x_bfgs, iters, f_bfgs, hist = bfgs_rank2_quadratic(Q, b, x0, tol=1e-10, max_iter=1000, enable_history=True)
print("BFGS (rank-2) Solution:", x_bfgs)
print("Iterations:", iters)
print("Final f(x):", f_bfgs)

x_exact = np.linalg.solve(Q, 0.5 * b)
print("Exact solution:", x_exact)
print("Error norm:", np.linalg.norm(x_bfgs - x_exact))

BFGS (rank-2) Solution: [ 5.00000000e-01  4.80432127e-14 -6.49722098e-14]
Iterations: 8
Final f(x): -0.375
Exact solution: [0.5 0.  0. ]
Error norm: 1.0472287683687152e-13
